In [45]:
import numpy as np
import pandas as pd
import random

In [46]:
class Grid:
    width = 0
    height = 0
    target = [-1, -1]
    current = [-1, -1]
    grid = []
    actions = [
     [0, -1], [0, 1], [1, 0], [-1, 0]
    ]
    directions = [
        [-1, 0], [-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1]
    ]
    path_blocks = None
    obstacles = None
    num_obstacles = 0
    
    
    NEG_REWARD = -10
    POS_REWARD = 200000
    MAX_DIST = -1
    
    def __init__(self, width, height, target_x, target_y, current_x = 0, current_y = 0, num_obstacles = 0, grid = None):
        self.grid = []
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        for i in range(height):
            self.grid.append([0] * width)
        
        self.target = [target_x, target_y]
        self.current  = [current_x, current_y]
        self.grid[self.target[0]][self.target[1]] = 2
        self.grid[self.current[0]][self.current[1]] = 1
        
        self.MAX_DIST = self.distance([0, 0], [self.height, self.width])
        self.path_blocks = self.findPath()
        self.obstacles = self.findObstacles()
        self.game_over = False
    
    def getObstacles(self):
        return repr(self.obstacles)
    
    def getState2(self):
        return [self.height, self.width, self.current, self.target]
        
    def getState(self):
        state = [0] * 16
        state1 = [0] * 16
        
        for i,direction in enumerate(self.directions):
            d, e = self.find(direction)
            state[8 * e + i] = d
            state1[8 * e + i] = 1
            
        return state + state1, self.current == self.target or self.game_over
    
    def distance(self, a, b):
        return (a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2
    
    def find(self, direction): # returns distance, type of entity (0 - wall, 1 - target)
        ni, nj = self.current[0] + direction[0], self.current[1] + direction[1]
        d = self.distance([ni, nj], self.current)
        while(not (ni < 0 or ni >= self.height or nj < 0 or nj >= self.width)):
            if(self.grid[ni][nj] == 2):
                return d / self.MAX_DIST, 1
            if(self.grid[ni][nj] == -1):
                return d / self.MAX_DIST, 0
            
            ni += direction[0]
            nj += direction[1]
            d = self.distance([ni, nj], self.current)
        return d / self.MAX_DIST, 0
        
    def move(self, action):
        action = self.actions[action]
        ni = self.current[0] + action[0]
        nj = self.current[1] + action[1]
        reward = 0
        
        if(ni < 0 or ni >= self.height or nj < 0 or nj >= self.width or self.grid[ni][nj] == -1):
            reward = self.NEG_REWARD
            self.game_over = True
        else:
            self.grid[self.current[0]][self.current[1]] = 0
            self.grid[ni][nj] = 1
            self.current = [ni, nj]
            reward = self.POS_REWARD if self.current == self.target else -1
        return self.getState(), reward
    
    def findPath(self):
        target = tuple(self.target)
        cur = tuple(self.current)
        q = [cur]
        parents = {}
        visited = set()
        found = False
        while(len(q) > 0 and found == False):
            size = len(q)
            random.shuffle(q)
            for i in range(size):
                u = q[0]
                del q[0]
                if(u in visited):
                    continue
                visited.add(u)
                for d in self.actions:
                    v = (u[0] + d[0], u[1] + d[1])
                    if(v[0] < 0 or v[1] < 0 or v[0] >= self.height or v[1] >= self.width):
                        continue
                    q.append(v)
                    if(v not in parents):
                        parents[v] = u
                    if v == target:
                        found = True
                if(found):
                    break         
        u = target
        ret = set()
        while(u in parents and u != cur):
            ret.add(u)
            u = parents[u]
        ret.add(u)
        return ret
    
    def findObstacles(self):
        num_obstacles = min(self.num_obstacles, (self.width * self.height - (self.width + self.height + 1)))
        ret = set()
        for i in range(num_obstacles):
            c = (random.randint(0, self.height - 1), random.randint(0, self.width - 1))
            if c not in self.path_blocks:
                ret.add(c)
        for pos in ret:
            self.grid[pos[0]][pos[1]] = -1
        return ret